<a href="https://colab.research.google.com/github/croco22/CapstoneProjectTDS/blob/seb/Generate_Flie%C3%9Ftext_Daten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extra Task: Continous Text generation

In [23]:
!pip install word2number

import json
import time
import google.generativeai as genai
from google.colab import userdata
import requests
from word2number import w2n
import re
import random  # Random-Modul importieren
import pandas as pd

# API setup
key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=key)
ai_model = genai.GenerativeModel('gemini-1.5-flash')

# Read dataset file
url = 'https://raw.githubusercontent.com/croco22/CapstoneProjectTDS/refs/heads/main/qa_dataset.json'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print("Retrieved file: qa_dataset.json")
else:
    print("Error while parsing a file: ", response.status_code)

Retrieved file: qa_dataset.json


In [24]:
# Neues Datenset generieren
def generate_dataset(data):
    new_dataset = []

    # Sicherstellen, dass keine Duplikate in Gruppen sind
    unique_questions = list({entry["question"]: entry for entry in data}.values())

    # Fragen mischen
    random.shuffle(unique_questions)

    # Gruppiere die Daten in Dreiergruppen
    for i in range(0, len(unique_questions), 3):
        group = unique_questions[i:i+3]

        # Überprüfen, ob die Gruppe vollständig ist (d.h. genau 3 Elemente enthält)
        if len(group) < 3:
            break

        # Fließtext generieren
        text_parts = []
        questions = []

        for entry in group:
            question = entry["question"]
            answer = entry["intended_answer"]
            context = entry["context"]

            text_parts.append(f"{context}")
            questions.append({
                "question": question,
                "intended_answer": answer
            })

        # Den Fließtext zusammenfügen
        text = " ".join(text_parts)

        # Eintrag in das neue Dataset hinzufügen
        new_dataset.append({
            "text": text,
            "questions": questions
        })

    return new_dataset

# Speichern des neuen Datensets
def save_dataset(data, output_path):
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


In [25]:
output_file = "new_qa_dataset.json"  # Neuer Dateipfad

# Neues Datenset generieren
new_dataset = generate_dataset(data)

# Neues Datenset speichern
save_dataset(new_dataset, output_file)
print(f"Das neue Dataset wurde in {output_file} gespeichert.")

Das neue Dataset wurde in new_qa_dataset.json gespeichert.


In [26]:
data = pd.read_json('new_qa_dataset.json')
data.head()

,text,questions
0,"Ah, notes, yes, a vital part of my life, I mus...","[{'question': 'Notes', 'intended_answer': None..."
1,"Finally, I find it so difficult to see the big...",[{'question': 'What specific challenges are yo...
2,"Okay, so given the choices, my reason for usin...",[{'question': 'Which of the following best des...
3,A CRM system you say? That makes me think imme...,"[{'question': 'CRM-System', 'intended_answer':..."
4,"Let me see, yes, the phone number you should u...",[{'question': 'What phone number can we use fo...


In [27]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [28]:
data["text"] = pipe(data["text"], max_length=80, min_length=30, do_sample=False)

ValueError:  `args[0]`: 0     Ah, notes, yes, a vital part of my life, I mus...
1     Finally, I find it so difficult to see the big...
2     Okay, so given the choices, my reason for usin...
3     A CRM system you say? That makes me think imme...
4     Let me see, yes, the phone number you should u...
5     So, basically, the company you're asking about...
6     Ultimately, my main focus for using this proje...
7     Under no circumstances do I agree to data proc...
8     I have a bit of a mish-mash of problems I need...
9     Yes, I can do that, no problem at all, my phon...
10    You know what would be just lovely for me? A w...
11    Alright, to proceed from here, the plan of act...
12    I'm here because I really need to check projec...
13    I'd say it’s realistically going to be done by...
14    "Happy to help with that! My business number i...
15    I'm afraid I can't really choose any of the pr...
16    My job duties aren't straightforward, and I te...
17    Let's see here, wh

In [ ]:
data.head()